# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: Sparsh Patel
Date: 08/16/2025

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [17]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
os.chdir("/Users/sparshpatel/bootcamp_sparsh_patel/homework/homework4/")

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


In [ ]:
env_path = pathlib.Path('/Users/sparshpatel/bootcamp_sparsh_patel/homework') / '.env'
load_dotenv(dotenv_path=env_path)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


Here I changed the environment path to get to the correct location so that the .env file can be accessed.

In [4]:
print(os.getcwd())

/Users/sparshpatel/bootcamp_sparsh_patel/homework/homework4


Making sure that I am in the correct directory.

## Helpers 

In [39]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull 
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [30]:
SYMBOL = 'BRK-B'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

KeyError: "['adj_close'] not in index"

In [25]:
print(js)


{'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}


### Here you can see that the adjusted close function has been shifted to the premium version of the alphavantage api, so we were not able to access that.

In [42]:
SYMBOL = 'BRK-A'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))

if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {
        'function':'TIME_SERIES_DAILY',
        'symbol':SYMBOL,
        'outputsize':'compact',
        'apikey': os.getenv('ALPHAVANTAGE_API_KEY')
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    
    # Use '4. close' instead of adjusted close
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(
        columns={'index':'date','4. close':'close'}
    )[['date','close']]
    df_api['date'] = pd.to_datetime(df_api['date'])
    df_api['close'] = pd.to_numeric(df_api['close'])

else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Close']]
    df_api.columns = ['date','close']

# Validate
v_api = validate(df_api, ['date','close'])
v_api


{'missing': [], 'shape': (100, 2), 'na_total': 0}

#### Successfully retrieved the ticker data.

In [43]:
print(js)

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'BRK-A', '3. Last Refreshed': '2025-08-15', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-08-15': {'1. open': '723080.0012', '2. high': '724040.0000', '3. low': '714246.3812', '4. close': '717250.0000', '5. volume': '310'}, '2025-08-14': {'1. open': '718026.4200', '2. high': '718680.0000', '3. low': '712114.8300', '4. close': '718680.0000', '5. volume': '269'}, '2025-08-13': {'1. open': '707525.8650', '2. high': '719339.4300', '3. low': '706702.9112', '4. close': '715490.4100', '5. volume': '443'}, '2025-08-12': {'1. open': '697051.1212', '2. high': '708299.9900', '3. low': '697051.1212', '4. close': '704700.1000', '5. volume': '338'}, '2025-08-11': {'1. open': '699742.1050', '2. high': '702189.6200', '3. low': '695680.0000', '4. close': '698440.0000', '5. volume': '478'}, '2025-08-08': {'1. open': '695837.4400', '2. high': '699940.9900', '3. lo

In [44]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data/raw/api_source-alpha_symbol-BRK-A_20250817-174318.csv


#### Successfully stored the fetched data into a CSV file in the data/raw folder.

## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [41]:
SCRAPE_URL = 'https://finance.yahoo.com/most-active'  
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

{'missing': [], 'shape': (25, 12), 'na_total': 25}

### Chose Yfinance tables. Some other tables had errors in their structures while scraping.

In [37]:
df_api.isna().sum()


date     0
close    0
dtype: int64

In [38]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data/raw/scrape_site-example_table-markets_20250817-170528.csv


#### Successfully stored the scraped table to a CSV file in the data/raw folder.

## Documentation
- Please go through the README file for proper documentation.